# Agricultural Pricing

## Objective and Prerequisites

This problem is an example of an agricultural pricing problem, where we want to determine the prices and  demand for the dairy products of a country in order to maximize total revenue derived from the sales of those products. This problem is formulated as a quadratic optimization problem using the Gurobi Python API and solved with the Gurobi Optimizer.

This model is example 21 from the fifth edition of Model Building in Mathematical Programming, by H. Paul Williams on pages 276-278 and 333-335.

This modeling example is at the intermediate level, where we assume that you know Python and are familiar with the Gurobi Python API. In addition, you should have some knowledge about building mathematical optimization models.


**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 

**Gurobi License** <br /> 
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an [evaluation license](https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-EDU-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_AGRICULTURAL_PRICING_COM_EVAL_GitHub&utm_term=Agricultural%20Pricing&utm_content=C_JPM) as a *commercial user*, or download a [free license](https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-EDU-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_AGRICULTURAL_PRICING_ACADEMIC_EVAL_GitHub&utm_term=Agricultural%20Pricing&utm_content=C_JPM) as an *academic user*.

## Problem Description

The government of a country wants to decide what prices should be charged for its dairy products: milk, butter, and cheese. All these products 
are produced 
(directly or indirectly) from the country’s raw milk production operations. This raw milk is divided into two main components: fat and dry matter. After subtracting the quantities of fat and dry matter, which are used for making products for export or consumption on the farms, there is a total yearly availability of 600,000 tons of fat and 750,000 tons of dry matter. This is all available for producing milk, butter and two kinds of cheese for domestic consumption. The percentage composition of the products are given in the following table:

| Composition | Fat (%) | Dry matter (%) |
| --- | --- | --- |
| Milk | 4 | 9 |
| Butter | 80 | 2 |
| Cheese 1 | 35 | 30 |
| Cheese 2 | 25 | 40 |

The table below shows last year's domestic consumption (demand) and prices for the dairy products:

| Dairy <br /> products | Milk | Butter | Cheese 1 | Cheese 2 |
| --- | --- | --- | --- | --- |
| Demand (1000 tons) | 4.82 | 0.32 |  0.21 | 0.07 |
| Price (dollars/ton) | 297 | 720 | 1050 | 815 |

The elasticities and cross-elasticities are given in the table below:

| Milk | Butter | Cheese 1 | Cheese 2 | Cheese 1 to  <br /> Cheese 2 |  Cheese 2 to  <br /> Cheese 1 |
| --- | --- | --- | --- | --- |  --- |
| 0.4 | 2.7 | 1.1 |  0.4 | 0.1 |  0.4 |

The price index cannot be raised higher than last year.This constraint establishes that the new prices must be such that the total cost of last year’s consumption would not be increased. 
Last year's price index is 1.939 (measured in thousand dollars).

The goal is to determine what prices and demand  maximizes the total revenue.

## Model Formulation

### Sets and Indices

$d \in \text{Dairy}=\{\text{milk}, \text{butter}, \text{cheese1}, \text{cheese2} \}$

$c \in \text{Components}=\{\text{fat}, \text{dry_matter} \}$

### Parameters

$\text{capacity}_{c} \in \mathbb{R}^+$: Yearly availability of component $c$ (1000 tons).

$\text{qtyper}_{c,d} \in [0,1]$: Percentage of component $c$ in dairy product $d$.

$\text{consumption}_{d} \in \mathbb{R}^+$: Last year domestic consumption of dairy product  $d$ (1000 tons).

$\text{price}_{d} \in \mathbb{R}^+$: Last year price of dairy product $d$ (dollars/1000 tons).

$\text{elasticity}_{d} \in \mathbb{R}^+$: Last year price elasticity of domestic consumption of dairy product $d$.

$\text{elasticity12} \in \mathbb{R}^+$: Last year price cross-elasticity of domestic consumption of cheese 1 and cheese 2.

$\text{elasticity21} \in \mathbb{R}^+$: Last year price cross-elasticity of domestic consumption of cheese 2 and cheese 1.

$\text{prcIndex} \in \mathbb{R}^+$: Price index reflecting last year total consumption cost.

### Decision Variables

$\text{p}_{d} \in \mathbb{R}^+$: Price of dairy product $d$ (dollars/1000 tons).

$\text{q}_{d} \in \mathbb{R}^+$: Demand of dairy product $d$ (1000 tons).

### Constraints

**Capacity**: The limited availabilities of fat and dry matter are enforced by the following constraints.

\begin{equation}
\sum_{d \in \text{Dairy}}{\text{qtyper}_{c,d}*\text{q}_{d} } \leq \text{capacity}_{c} \quad \forall c \in \text{Components}
\end{equation}


**Price index**: This constraint establishes that the new prices must be such that the total cost of last year’s consumption would not be increased.

\begin{equation}
\sum_{d \in \text{Dairy}}{\text{consumption}_{d}*\text{p}_{d} } \leq \text{prcIndex}
\end{equation}

**Elasticities**: The demand variables $q_{d}$ are related to the price variables $p_{d}$  through the price elasticities relationships. We approximate the elasticities with linear relationships.

Milk elasticity.
$$
(\text{q}_{milk} - \text{consumption}_{milk})/\text{consumption}_{milk}) = -\text{elasticity}_{milk}*(\text{p}_{milk} - \text{price}_{milk})/\text{price}_{milk})
$$

Butter elasticity.
$$
(\text{q}_{butter} - \text{consumption}_{butter})/\text{consumption}_{butter}) = -\text{elasticity}_{butter}*(\text{p}_{butter} - \text{price}_{butter})/\text{price}_{butter})
$$

Cheese 1 elasticity.
$$
(\text{q}_{cheese1} - \text{consumption}_{cheese1})/\text{consumption}_{cheese1}) = -\text{elasticity}_{cheese1}*(\text{p}_{cheese1} - \text{price}_{cheese1})/\text{price}_{cheese1}) 
$$

$$
+ elasticity12*(\text{p}_{cheese2} - \text{price}_{cheese2})/\text{price}_{cheese2})
$$

Cheese 2 elasticity.
$$
(\text{q}_{cheese2} - \text{consumption}_{cheese2})/\text{consumption}_{cheese2}) = -\text{elasticity}_{cheese2}*(\text{p}_{cheese2} - \text{price}_{cheese2})/\text{price}_{cheese2}) 
$$

$$
+ elasticity21*(\text{p}_{cheese1} - \text{price}_{cheese1})/\text{price}_{cheese1}) 
$$

### Objective Function

**Revenue**: The objective is to maximize total revenue.

\begin{equation}
\text{Maximize} \quad \sum_{d \in \text{Dairy}}{\text{q}_{d}*\text{p}_{d} }
\end{equation}

---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [1]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0

## Input data

We define all the input data for the model.

In [2]:
# List of dairy products.

dairy = ['milk', 'butter', 'cheese1', 'cheese2']

components = ['fat', 'dryMatter']


# Create a dictionary to capture the percentage composition of the products.

cd, qtyper = gp.multidict({
    ('fat','milk'): 0.04,
    ('fat','butter'): 0.8,
    ('fat','cheese1'): 0.35,
    ('fat','cheese2'): 0.25,
    ('dryMatter','milk'): 0.09,
    ('dryMatter','butter'): 0.02,
    ('dryMatter','cheese1'): 0.3,
    ('dryMatter','cheese2'): 0.4
})

# Create a dictionary to capture the yearly availability of components (1000 tons).

components, capacity = gp.multidict({
    ('fat'): 600,
    ('dryMatter'): 750
})

# Create a dictionary to capture last year's domestic consumption and prices

dairy, consumption, price, elasticity = gp.multidict({
    ('milk'): [4.82, 0.297, 0.4],
    ('butter'): [0.32, 0.72, 2.7],
    ('cheese1'): [0.21, 1.05, 1.1],
    ('cheese2'): [0.07, 0.815, 0.4]
})

elasticity12 = 0.1
elasticity21 = 0.4

priceIndex = 1.939

## Model Deployment

We create a model and the variables. The decision variables of this model are the prices and demand for the dairy products. 

Solving bilinear problems with Gurobi is as easy as configuring the global parameter `nonConvex`, and setting this parameter to the value of 2.

In [3]:
model = gp.Model('AgriculturalPricing')

# Set global parameters. 
model.params.nonConvex = 2

# Quantity of dairy products.
qvar = model.addVars(dairy, name="qvar")

# Price of dairy products.
pvar = model.addVars(dairy, name="pvar")


Using license file c:\gurobi\gurobi.lic
Set parameter TokenServer to value SANTOS-SURFACE-
Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1


The limited availabilities of fat and dry matter are enforced by the following constraints.

In [4]:
# Capacity constraint.

fatCap = model.addConstrs( (gp.quicksum(qtyper[c,d]*qvar[d] for d in dairy) <= capacity[c] for c in components  ), 
                          name='fatCap')

This constraint ensures that the new prices must be such that the total cost of last year’s consumption would not be increased.

In [5]:
# Price index constraint.

priceIndex = model.addConstr( (gp.quicksum(consumption[d]*pvar[d] for d in dairy) <= priceIndex ), name='priceIndex')

The demand variables are related to the price variables  through the price elasticities relationships. We approximate the elasticities with linear relationships.

In [6]:
# Elasticity constraints

elasMilk = model.addConstr( (qvar['milk']-consumption['milk'])/consumption['milk']  
                           == -elasticity['milk']*(pvar['milk']-price['milk'])/price['milk'], name='elasMilk')

elasButter = model.addConstr( (qvar['butter']-consumption['butter'])/consumption['butter']  
                           == -elasticity['butter']*(pvar['butter']-price['butter'])/price['butter'], name='elasButter')

elasCheese1 = model.addConstr( (qvar['cheese1']-consumption['cheese1'])/consumption['cheese1']  
                           == -elasticity['cheese1']*(pvar['cheese1']-price['cheese1'])/price['cheese1']
                              +elasticity12*(pvar['cheese2']-price['cheese2'])/price['cheese2'] , name='elasCheese1')

elasCheese2 = model.addConstr( (qvar['cheese2']-consumption['cheese2'])/consumption['cheese2']  
                           == -elasticity['cheese2']*(pvar['cheese2']-price['cheese2'])/price['cheese2']
                              +elasticity21*(pvar['cheese1']-price['cheese1'])/price['cheese1'] , name='elasCheese2')

The objective function is to maximize revenue.

In [7]:
# Quadratic objective function.

obj = gp.quicksum(qvar[d]*pvar[d] for d in dairy)

model.setObjective(obj, GRB.MAXIMIZE)

In [8]:
# Verify model formulation

model.write('AgriculturalPricing.lp')

# Run optimization engine

model.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 7 rows, 8 columns and 22 nonzeros
Model fingerprint: 0x4fffdf2e
Model has 4 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-02, 1e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+02]
Presolve removed 2 rows and 0 columns

Continuous model is non-convex -- solving as a MIP.

Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 14 rows, 13 columns, 36 nonzeros
Presolved model has 4 bilinear constraint(s)
Variable types: 13 continuous, 0 integer (0 binary)

Root relaxation: objective 2.791207e+00, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.79121    0    4          -    2.79121      -     -    0s
H    0     0                       2.0

## Analysis

The table below shows the price (dollars/ton) and demand (tons) at equilibrium, for each dairy product. The total revenue generated is $\$ 2,067,170,173$.

In [9]:
# Output Report
price_demand = pd.DataFrame(columns=["Products", "Price", "Demand"])
for d in dairy:
    price_demand = price_demand.append({"Products": d, "Price": '${:,.2f}'.format(round(1000*pvar[d].x)), "Demand": '{:,.2f}'.format(round(1e6*qvar[d].x))}, ignore_index=True)  
price_demand.index=[''] * len(price_demand)
price_demand

,Products,Price,Demand
,milk,$322.00,"4,658,539.00"
,butter,$422.00,"677,741.00"
,cheese1,$832.00,"265,745.00"
,cheese2,"$1,113.00","53,921.00"


---
## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright © 2020 Gurobi Optimization, LLC